这个实验要求我们利用逻辑回归识别0-9手写数字。很明显，这是一个多分类问题。

由于数据集是MATLAB的本机格式，所以我们要使用python的loadmat来读取数据集

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat

In [2]:
data = loadmat('ex3data1.mat')
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [3]:
data['X'].shape, data['y'].shape

((5000, 400), (5000, 1))

我们可以看到，数据集有5000个样本组成，每张图片是$20\times20$的像素图片，被压维成了长为400的一维向量
像素图片中每个点是一个浮点数，指明这个位置的**灰度值**。在标签y中，用0表示数字10，1-9正常表示数字1-9

定义sigmoid函数：

In [4]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

逻辑回归的代价函数为：$J\left( \theta \right)=\frac{1}{m}\sum\limits_{i=1}^{m}{\Big[-{{y}^{(i)}}\log \left( {h_\theta}\left( {{x}^{(i)}} \right) \right)-\left( 1-{{y}^{(i)}} \right)\log \left( 1-{h_\theta}\left( {{x}^{(i)}} \right) \right) \Big]} + \frac{\lambda}{2m}\sum_{j = 1}^{n}\theta_j^2 $

In [17]:
def cost(theta, X, y, lamb):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)

    first = np.multiply(-y, np.log(sigmoid(X * theta.T)))
    second = np.multiply(y - 1, np.log(1 - sigmoid(X * theta.T)))
    reg = lamb / (2 * len(X)) * np.sum(np.power(theta[:, 1 : theta.shape[1]], 2))
    return np.sum(first + second) / len(X) + reg

梯度下降:

${{\theta }_{0}}:={{\theta }_{0}}-\alpha\frac{1}{m}\sum\limits_{i=1}^{m}{({{h}_{\theta }}({{x}^{(i)}})-{{y}^{(i)}})}x_{0}^{(i)} \\$
${{\theta }_{j}}:={{\theta }_{j}}-\alpha \Big[ \frac{1}{m}\sum\limits_{i=1}^{m}{({{h}_{\theta }}({{x}^{(i)}})-{{y}^{(i)}})}x_{j}^{(i)} + \frac{\lambda}{m}\theta_j \Big] \; , \; for \; j = 1,2,3,...,n$

In [6]:
def gradientDescent(theta, X, y, lamb):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)

    n = int(theta.ravel().shape[1])
    grad = np.zeros(n)
    err = sigmoid(X * theta.T) - y

    for i in range(n):
        term = np.multiply(err, X[:, i])

        if i == 0:
            grad[i] = np.sum(term) / len(X)
        else:
            grad[i] = np.sum(term) / len(X) + lamb / len(X) * theta[:, i]
    
    return grad

上面是我们之前写的梯度下降的普通版本，其实我们还可以将其改写为**向量化**的版本：

In [7]:
def gradient(theta, X, y, lamb):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)

    n = int(theta.ravel().shape[1])
    err = sigmoid(X * theta.T) - y
    grad = (X.T * err).T / len(X) + lamb / len(X) * theta
    grad[0, 0] = np.sum(err.T * X[:, 0 : 1]) / len(X)

    return np.array(grad).ravel()

接下来我们构建分类器，由于这个任务有10个可能的类别，所以我们采用“一对多”的分类方法。

我们将样本对每个类别进行梯度下降，得到10个局部最小$\theta$，最后通过$h_{\theta}(x)$运算，取最大可能性的那个类别作为预测结果

In [18]:
from scipy.optimize import minimize

def one_vs_all(X, y, num_labels, lamb):
    m = X.shape[0]
    n = X.shape[1]

    # num_labels * (n + 1) matrix for each tags' theta
    all_theta = np.zeros((num_labels, n + 1))

    # add the bias term
    X = np.insert(X, 0, values=np.ones(m), axis=1)

    for i in range(1, num_labels + 1):
        theta = np.zeros(n + 1)
        y_i = np.array([1 if label == i else 0 for label in y])
        y_i = np.reshape(y_i, (m, 1))

        # minimize the objective function
        fmin = minimize(fun=cost, x0=theta, args=(X, y_i, lamb), method='TNC', jac=gradient)
        all_theta[i - 1, :] = fmin.x
    
    return all_theta

我们查看y中的分类标签，确保与我们的想法一致

In [9]:
np.unique(data['y'])

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8)

In [19]:
all_theta = one_vs_all(data['X'], data['y'], 10, 1)
all_theta

array([[-2.38284954e+00,  0.00000000e+00,  0.00000000e+00, ...,
         1.30507327e-03, -9.06387081e-10,  0.00000000e+00],
       [-3.18224985e+00,  0.00000000e+00,  0.00000000e+00, ...,
         4.45533792e-03, -5.07956639e-04,  0.00000000e+00],
       [-4.79877410e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -2.87335685e-05, -2.47343575e-07,  0.00000000e+00],
       ...,
       [-7.98722034e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -8.94700016e-05,  7.21374762e-06,  0.00000000e+00],
       [-4.56958867e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.33747010e-03,  1.00048086e-04,  0.00000000e+00],
       [-5.40437806e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.16589020e-04,  7.87959477e-06,  0.00000000e+00]])

完成了对于所有类别的梯度下降后，我们现在进行预测

In [20]:
def predict(X, all_theta):
    m = X.shape[0]
    n = X.shape[1]
    num_labels = all_theta.shape[0]
    
    X = np.insert(X, 0, values=np.ones(m), axis=1)
    X = np.matrix(X)
    all_theta = np.matrix(all_theta)

    # calculate the probability of each tags
    h = sigmoid(X * all_theta.T)

    h_max = np.argmax(h, axis=1) # 0-indexed

    return h_max + 1

ok, 使用predict函数为每个样本预测其所属的类别。

In [22]:
y_pred = predict(data['X'], all_theta)
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, data['y'])]
accuracy = sum(correct) / len(correct) * 100
print('accuracy = {0}%.'.format(accuracy))

accuracy = 94.46%.
